# import

In [24]:
import pandas as pd
import numpy as np
from pyecharts import options as opts
from pyecharts.charts import BMap
from pyecharts.globals import ChartType, SymbolType ,BMapType
import folium
import os
import datetime as dt
from scipy.spatial.distance import pdist, squareform
from geopy.distance import distance
import itertools
import json
import polyline
import time
from time import perf_counter
import requests

np.random.seed(0)
pd.set_option('display.max_rows', 120)

# 数据导入

In [25]:
def get_minus(now):#获得分钟
    return int(now.split(":")[0]) * 60 + int(now.split(":")[1])
def get_time(minutes,now="00:00"):#获得时间格式
    return (dt.datetime.strptime(now, '%H:%M') + dt.timedelta(minutes=int(minutes)) ).strftime('%H:%M')

In [26]:
df=pd.read_excel('表2-11 POI信息表.xlsx')
df['open_time']=df['open_time'].apply(lambda x:eval(x))#open_time是字符串，将其变成可执行对象
recommend=pd.read_csv('recommend.csv')#每个城市能旅游几天
df_time_matrix=pd.read_csv('TimeMatrix.txt',header=None,sep=' ')
df_dis_matrix=pd.read_csv('DisMatrix.txt',header=None,sep=' ')

df_polyline=pd.read_csv('表2-13 outerCity.csv')
df_polyline['outerCity']=df_polyline['outerCity'].apply(lambda x:eval(x))
df_polyline['points']=df_polyline['points'].apply(lambda x:eval(x))

hotel_num=np.sum(df.type==0)#酒店数量
time_matrix=df_time_matrix.to_numpy()
dis_matrix=df_dis_matrix.to_numpy()

# 上午开始时间
AM_Start = '08:00'
# 上午结束时间
AM_End = '11:30'
# 下午开始时间
PM_Start = '13:00'
# 下午结束时间
PM_End = '17:30'
AM_Start=get_minus(AM_Start)
AM_End=get_minus(AM_End)
PM_Start=get_minus(PM_Start)
PM_End=get_minus(PM_End)

df.loc[df.type==0,'stay_time']=60
df.loc[df.type=='SC','stay_time']=np.random.choice(np.arange(60,130,10), size=sum(df.type=='SC'), replace=True)#允许重复采样
df.loc[df.type2=='SC1','stay_time']=np.random.choice(np.arange(120,190,10), size=sum(df.type2=='SC1'), replace=True)
df.loc[df.type=='SN','stay_time']=np.random.choice(np.arange(120,190,10), size=sum(df.type=='SN'), replace=True)
df.loc[df.type=='SP','stay_time']=np.random.choice(np.arange(60,130,10), size=sum(df.type=='SP'), replace=True)

In [27]:
cityname=['Auckland','Tauranga','Taupo']
for i in cityname:
    temp1=df.loc[(df.type==1)&(df.city==i),'innerCity']
    print(f'{i}景点在城内的比例：{sum(temp1)}/{len(temp1)}={sum(temp1)/len(temp1)}')
    temp2=df.loc[(df.type==0)&(df.city==i),'innerCity']
    print(f'{i}酒店在城内的比例：{sum(temp2)}/{len(temp2)}={sum(temp2)/len(temp2)}')

Auckland景点在城内的比例：14/117=0.11965811965811966
Auckland酒店在城内的比例：14/30=0.4666666666666667
Tauranga景点在城内的比例：16/36=0.4444444444444444
Tauranga酒店在城内的比例：7/14=0.5
Taupo景点在城内的比例：13/25=0.52
Taupo酒店在城内的比例：14/15=0.9333333333333333


## city council

In [28]:
council={'Alexandra': {'lat': -45.2478722, 'lng': 169.3843742},
 'Aoraki/Mount Cook': {'lat': -43.735565, 'lng': 170.1010159},
 'Auckland': {'lat': -36.8508852, 'lng': 174.7645088},
 'Blenheim': {'lat': -41.5135267, 'lng': 173.9597988},
 'Cape Reinga': {'lat': -34.5046644, 'lng': 172.7975555},
 'Christchurch': {'lat': -43.5320213, 'lng': 172.6306049},
 'Collingwood': {'lat': -40.6809989, 'lng': 172.6803683},
 'Cromwell': {'lat': -45.0459389, 'lng': 169.1954436},
 'Dargaville': {'lat': -35.9412954, 'lng': 173.8685666},
 'Dunedin': {'lat': -45.8795431, 'lng': 170.5005866},
 'Franz Josef/Waiau': {'lat': -43.387351, 'lng': 170.1833513},
 'Geraldine': {'lat': -44.0922941, 'lng': 171.2446511},
 'Gisborne': {'lat': -38.664151, 'lng': 178.0228231},
 'Gore': {'lat': -46.1040965, 'lng': 168.9421808},
 'Greymouth': {'lat': -42.4614298, 'lng': 171.1985066},
 'Haast': {'lat': -43.8600115, 'lng': 169.0455549},
 'Hamilton': {'lat': -37.7824869, 'lng': 175.2527451},
 'Hicks Bay': {'lat': -37.5921434, 'lng': 178.2917021},
 'Invercargill': {'lat': -46.4180089, 'lng': 168.3614579},
 'Kaikoura': {'lat': -42.3994557, 'lng': 173.6799136},
 'Kaitaia': {'lat': -35.1088064, 'lng': 173.2587128},
 'Lake Tekapo': {'lat': -44.0060459, 'lng': 170.4793282},
 'Masterton': {'lat': -40.9461683, 'lng': 175.6672805},
 'Milford Sound': {'lat': -44.6681441, 'lng': 167.9272583},
 'Murchison': {'lat': -41.8000778, 'lng': 172.3254732},
 'Napier': {'lat': -39.5108318, 'lng': 176.8761601},
 'Nelson': {'lat': -41.2985372, 'lng': 173.2444149},
 'New Plymouth': {'lat': -39.0571644, 'lng': 174.0794107},
 'Oamaru': {'lat': -45.0965906, 'lng': 170.9714359},
 'Paihia': {'lat': -35.2821179, 'lng': 174.091043},
 'Palmerston North': {'lat': -40.35442159999999, 'lng': 175.6098077},
 'Picton': {'lat': -41.2954611, 'lng': 174.0028053},
 'Queenstown': {'lat': -45.0301798, 'lng': 168.6614735},
 'Rotorua': {'lat': -38.1445895, 'lng': 176.2377732},
 'Taumarunui': {'lat': -38.8832802, 'lng': 175.2563523},
 'Taupo': {'lat': -38.6842631, 'lng': 176.0704678},
 'Tauranga': {'lat': -37.6869679, 'lng': 176.1654268},
 'Te Anau': {'lat': -45.4144692, 'lng': 167.7180308},
 'Thames': {'lat': -37.1383451, 'lng': 175.5405341},
 'Timaru': {'lat': -44.3903891, 'lng': 171.2372696},
 'Twizel': {'lat': -44.2614876, 'lng': 170.0876395},
 'Waikaremoana': {'lat': -38.7788547, 'lng': 177.1289016},
 'Waitomo Caves': {'lat': -38.2606825, 'lng': 175.1113228},
 'Wanaka': {'lat': -44.6942959, 'lng': 169.1417463},
 'Wellington': {'lat': -41.2923807, 'lng': 174.7787471},
         
 'Westport': {'lat': -41.7545765, 'lng': 171.6059767},
 'Whakapapa Village': {'lat': -39.2005697, 'lng': 175.5393209},
 'Whakatane': {'lat': -37.9585448, 'lng': 176.9822232},
 'Whanganui': {'lat': -39.9328918, 'lng': 175.0286215},
 'Whangarei': {'lat': -35.7274778, 'lng': 174.3165559}}

council1=council.copy()
available_city=['Wellington','Christchurch','Dunedin','Napier','Rotorua','Invercargill','Queenstown','Gisborne','Palmerston North','Tauranga','Whanganui','Whangarei','Oamaru','Auckland','New Plymouth','Taupo','Whakatane','Alexandra','Blenheim','Geraldine','Gore','Greymouth','Kaikoura','Nelson','Wanaka']
for i in council:
    if i not in available_city:
        council1.pop(i)

In [29]:
# 城内景点的标准站暂定为4km以内
# df['innerCity2']=0
# for i,j in df[['lat','lng']].iterrows():
#     for k in council.values():
#         dis=distance((k['lat'],k['lng']),j).km
#         if dis<4: #具体可以修改
#             df.loc[i,'innerCity2']=1

# drawPicture()

In [30]:
# 根据序列构造出画直线图所需要的数据结构，根据pyechart
line = lambda li: [(li[i], li[i + 1]) for i in range(len(li) - 1)]
line2 = lambda li: [tuple(df.loc[i, ['lng', 'lat']].values.tolist()) for i in line(li)]

# [([176.2529811, -38.1429921], [176.2745111, -38.1550414]),
#  ([176.2745111, -38.1550414], [176.273631, -38.155665])]
# 全部点都画上
def drawPicture(li, name,time):
    bmap = BMap(init_opts=opts.InitOpts(width="2000px", height="1000px"))
    bmap.add_schema(
        baidu_ak=baidu_ak,
        center=[176.2377732, -38.1445895],  #所有点的中心
        is_roam=True,
        zoom=10)
    idx = 0
    hotel = []
    for val in df[df.type == 0].values:
        bmap.add_coordinate(f'{idx}', val[2], val[3])
        hotel.append([str(idx), val[0] + ' ' + val[1]])
        idx += 1
    scenery = []
    for val in df[df.type == 1].values:
        bmap.add_coordinate(f'{idx}', val[2], val[3])
        scenery.append([str(idx), val[0] + ' ' + val[1]])
        idx += 1
    restaurant = []
    for val in df[df.type == 2].values:
        bmap.add_coordinate(f'{idx}', val[2], val[3])
        restaurant.append([str(idx), val[0] + ' ' + val[1]])
        idx += 1
    cil = []
    for i, j in council1.items():
        bmap.add_coordinate(f'{idx}', j['lng'], j['lat'])
        cil.append([
            str(idx),
            i + ' ' + str(recommend.loc[recommend.city == i, 'day'].values[0])
        ])
        idx += 1

    bmap.add(
        type_="effectScatter",
        series_name="hotel",
        data_pair=hotel,
        symbol_size=3,
        is_large=True,  #图贴别多的时候开启
        itemstyle_opts=opts.ItemStyleOpts(color="red"),
        #         label_opts=opts.LabelOpts(formatter="{b}", position="right", is_show=1)
    )
    bmap.add(
        type_="effectScatter",
        series_name="scenery",
        data_pair=scenery,
        symbol_size=3,
        is_large=True,  #图贴别多的时候开启
        itemstyle_opts=opts.ItemStyleOpts(color="green"),
        #         label_opts=opts.LabelOpts(formatter="{b}", position="right", is_show=1)
    )
    bmap.add(
        type_="effectScatter",
        series_name="restaurant",
        data_pair=restaurant,
        symbol_size=3,
        is_large=True,  #图贴别多的时候开启
        itemstyle_opts=opts.ItemStyleOpts(color="blue"),
        #         label_opts=opts.LabelOpts(formatter="{b}", position="right", is_show=1)
    )
    bmap.add(
        type_="effectScatter",
        series_name="council",
        data_pair=cil,
        symbol_size=5,
        itemstyle_opts=opts.ItemStyleOpts(color="black"),
        #         label_opts=opts.LabelOpts(formatter="{b}", position="right", is_show=1)
    )
    #position 标签在点在右边，is_show不自动显示标签， {a}（系列名称），{b}（数据名称），{c}（数值数组）, {d}（无）
    for idx, val in enumerate(splitTour_day(li,time)):
        bmap.add(
            series_name=f'line{idx}',
            data_pair=line2(val),
            type_='lines',
            is_polyline=True,  #是否是多段线，在画lines图情况下
            is_large=True,  #图贴别多的时候开启
            effect_opts=opts.EffectOpts(symbol=SymbolType.ARROW,
                                        symbol_size=5,
                                        color="yellow"),
            linestyle_opts=opts.LineStyleOpts(opacity=1, width=1,
                                              curve=0.2)  # 线样式配置项
        )

#     bmap.set_series_opts(label_opts=opts.LabelOpts(formatter="{b}", position="right", is_show=1))
    bmap.set_global_opts(title_opts=opts.TitleOpts(title="New Zealand"))

    bmap.render(name + '.html')

#只把出现的点打上
def drawPicture2(li, name ,time):
    bmap = BMap(init_opts=opts.InitOpts(width="2000px", height="1000px"))
    bmap.add_schema(
        baidu_ak=baidu_ak,
        center=df.loc[li[0],['lng','lat']].tolist(),  #旅途第一个个点当成地图中心
        is_roam=True,
        zoom=10
    )
    temp = df.iloc[li, 0:6]
    hotel = []
    for idx, val in temp[temp.type == 0].drop_duplicates(subset=['city', 'name']).iterrows():
        bmap.add_coordinate(f'{idx}', val[2], val[3])
        hotel.append([str(idx), val[0] + ' ' + val[1]])
    scenery = []
    for idx, val in temp[temp.type == 1].drop_duplicates(subset=['city', 'name']).iterrows():
        bmap.add_coordinate(f'{idx}', val[2], val[3])
        scenery.append([str(idx), val[0] + ' ' + val[1]])
    restaurant = []
    for idx, val in temp[temp.type == 2].drop_duplicates(subset=['city', 'name']).iterrows():
        bmap.add_coordinate(f'{idx}', val[2], val[3])
        restaurant.append([str(idx), val[0] + ' ' + val[1]])
    cil = []

    for i, j in council1.items():
        bmap.add_coordinate(i, j['lng'], j['lat'])
        cil.append([i,i + ' ' + str(recommend.loc[recommend.city == i, 'day'].values[0])])


    #hotel
    bmap.add(
        type_="effectScatter",
        series_name="hotel",
        data_pair=hotel,
        symbol_size=3,
        is_large=True,  #图贴别多的时候开启
        itemstyle_opts=opts.ItemStyleOpts(color="red"),
        #         label_opts=opts.LabelOpts(formatter="{b}", position="right", is_show=1)
    )
    #scenery
    bmap.add(
        type_="effectScatter",
        series_name="scenery",
        data_pair=scenery,
        symbol_size=6,
        is_large=True,  #图贴别多的时候开启
        itemstyle_opts=opts.ItemStyleOpts(color="green"),
        #         label_opts=opts.LabelOpts(formatter="{b}", position="right", is_show=1)
    )
    #restaurant
    bmap.add(
        type_="effectScatter",
        series_name="restaurant",
        data_pair=restaurant,
        symbol_size=6,
        is_large=True,  #图贴别多的时候开启
        itemstyle_opts=opts.ItemStyleOpts(color="blue"),
        #         label_opts=opts.LabelOpts(formatter="{b}", position="right", is_show=1)
    )
    #council
    bmap.add(
        type_="effectScatter",
        series_name="council",
        data_pair=cil,
        symbol_size=3,
        itemstyle_opts=opts.ItemStyleOpts(color="black"),
        #         label_opts=opts.LabelOpts(formatter="{b}", position="right", is_show=1)
    )
    

    #线
    for idx, val in enumerate(splitTour_day(li,time)):
        arr = []
        for i in range(len(val) - 1):
            temp = polyline.decode(dict[f'{val[i]}_{val[i+1]}'])
            arr += [(i[1], i[0]) for i in temp]
        bmap.add(
            series_name=f'day{idx+1}',
            data_pair=line(arr),
            type_='lines',
            is_polyline=True,  #是否是多段线，在画lines图情况下
            is_large=True,  #图贴别多的时候开启
            effect_opts=opts.EffectOpts(symbol=SymbolType.ARROW,
                                        symbol_size=5,
                                        color="yellow",
                                        trail_length=0.5),
            linestyle_opts=opts.LineStyleOpts(opacity=0.6,
                                              width=2,
                                              curve=0.2)  # 线样式配置项
        )

    bmap.add_control_panel(
        copyright_control_opts=opts.BMapCopyrightTypeOpts(position=3),
        maptype_control_opts=opts.BMapTypeControlOpts(type_=BMapType.MAPTYPE_CONTROL_DROPDOWN),
        scale_control_opts=opts.BMapScaleControlOpts(),
        overview_map_opts=opts.BMapOverviewMapControlOpts(is_open=True),
        navigation_control_opts=opts.BMapNavigationControlOpts(),
        geo_location_control_opts=opts.BMapGeoLocationControlOpts(),
    )
    bmap.set_series_opts(label_opts=opts.LabelOpts(formatter="{b}", position="right", is_show=1))
    #position 标签在点在右边，is_show不自动显示标签， {a}（系列名称），{b}（数据名称），{c}（数值数组）, {d}（无）
    bmap.set_global_opts(title_opts=opts.TitleOpts(title="New Zealand"))
    bmap.render(name + '.html')


# 两点间的轨迹

In [31]:
# 画图时使用
# 用过的储存起来，下次用更快
with open('line.json', 'r', encoding='utf-8') as f:
    dict = json.load(f)

In [32]:
#如果没出现过，就从网上下载保存到line中
def get_line(Tour):
    for i in range(len(Tour) - 1):
        if f'{Tour[i]}_{Tour[i+1]}' in dict:
            continue
        strr = polyline.encode(df.loc[Tour[i:i + 2],['lat', 'lng']].values.tolist())
        url = f'http://router.project-osrm.org/route/v1/driving/polyline({strr})'
        response = requests.get(url).json()
        dict[f'{Tour[i]}_{Tour[i+1]}'] = response['routes'][0]['geometry']
    with open('line.json', 'w', encoding='utf-8') as f:
        json.dump(dict, f)

# 距离计算及合法解判断

In [33]:
def calcTime(li:list):#开车时间和停留时间相加
    time = [ time_matrix[ li[i] , li[i+1] ] + df.loc[li[i+1],'stay_time'] for i in range(len(li) - 1) ] #估算
    return np.sum(time)+AM_Start
def calcDistance(li:list):#计算出一个路程总距离
    dis = [ dis_matrix[ li[i] , li[i+1] ] for i in range(len(li) - 1) ]
    return np.sum(dis)

def splitTour_day(li,time):
    li=splitTour(li)
    split=[]
    i=0
    while i<len(li):
        if time+calcTime(li[i])+60<=PM_End and i+1!=len(li):
            split.append( li[i]+li[i+1][1:] )
            i+=2
            time=0
        else:
            split.append( li[i] )
            i+=1
            time=0
    return split

def splitTour(li):  #以酒店为点开始分割
    H_index = np.where(li < hotel_num)[0].tolist()
    return [li[H_index[i]:H_index[i + 1] + 1] for i in range(len(H_index) - 1)]

def combineTrip(li):#将多个trip合成tour
#     temp=sum(li,[])#将所有列表展开
    return [k for k,_ in itertools.groupby(sum(li,[]))]#删除相邻的数字

def feasible(li,week,start_time):#判断是否可行
    x=dailySchedule(li,week,start_time)
    for i,v in x[['open_time','arrive_time','leave_time']].iterrows():
        val=[get_minus(k) for k in v[0].split('-')]
        arr=[get_minus(v[1]),get_minus(v[2])]
        if not(val[0]<=arr[0] and arr[1]<=val[1]):
            return False
    return True

def dailySchedule(Tour,week,time):
    x=df.loc[Tour,['city','name', 'score','cost' ,'type1', 'type2','stay_time']]
    t_list={'arrive':['00:00'],
            'drive':[0],
            'leave':[time],#10分钟从机场到酒店,60分钟酒店休整,50分钟酒店吃饭
            'open_time':[ df.loc[Tour[0],'open_time'][week%7] ] }
    time=get_minus(t_list['leave'][0])
    for arr in splitTour_day(Tour,time):
        for i in range(len(arr)-1):
            t_list['open_time'].append(df.loc[arr[i+1],'open_time'][week%7])
            t_list['drive'].append(time_matrix[ arr[i], arr[i+1] ])
            time+=time_matrix[ arr[i], arr[i+1] ]

            t_list['arrive'].append(get_time(time))

            if arr[i+1]<hotel_num and calcTime(arr[0:i+2]) > AM_End:
                time+=50
            time+=df.loc[arr[i+1],'stay_time']
            t_list['leave'].append(get_time(time))
        time=AM_Start
        week+=1
        t_list['leave'][-1]=get_time(AM_Start)#一天的最后一个点是酒店，他的离开时间其实是第二天早上八点
#     t_list['leave'][-1]='00:00'
    x['arrive_time']=t_list['arrive']
    x['drive_time']=t_list['drive']
    x['leave_time']=t_list['leave']
    x['open_time']=t_list['open_time']
    
    return x.loc[:,['city','name','score','cost', 'type1', 'type2', 'open_time', 'drive_time','arrive_time', 'stay_time','leave_time']]

# dayTrip

In [34]:
def restaurantSelection(Tour:list,name:str,week:int,time:int,flag_r:int):
    restaurant=df.loc[(df.city==name)&(df.innerCity==1)&(df.type==2),:].index.tolist()
    if flag_r==0:
        restaurant=[i for i in restaurant if df.loc[i,'type1']!='RN']
    elif flag_r==1:
        restaurant=[i for i in restaurant if df.loc[i,'type1']!='RW']
    elif flag_r==2:
        restaurant=[i for i in restaurant if df.loc[i,'type1']!='RA']
    elif flag_r==3:
        restaurant=[i for i in restaurant if df.loc[i,'type1']!='RC']
        
    restaurant=list(set(restaurant)-set(Tour))#去除已经走过的点
    next_restaurant = np.argsort(dis_matrix[Tour[-1], restaurant])#距离最短的序列
    restaurant=[restaurant[i] for i in next_restaurant]#按照距离最短排序
    

    for i in restaurant:
        val=[get_minus(k) for k in df.loc[i,'open_time'][week%7].split('-')]
        arrive=time+time_matrix[Tour[-1],i]
        leave=arrive+df.loc[i,'stay_time']
#         现有时间+路程时间在第一个景点开放时间之内
        if  val[0]<arrive and leave<val[1]:
            time=leave
            Tour.append(i)
            flag_r=['RN','RW','RA','RC'].index(df.loc[i,'type1'])
            return Tour,time,flag_r
    #循环中，没有可行解，解除类型约束，即令flag_r=-1
    flag_r=-1
    return Tour,time,flag_r

In [35]:
def scenicSpotSelection(Tour:list,name:str,week:int,time:int,flag_s:int):
    scenicSpot=df.loc[(df.city==name)&(df.innerCity==1)&(df.type==1),:].index.tolist()
#     scenicSpot=df.loc[(df.city==name)&(df.innerCity2==1)&(df.type==1),:].index.tolist()
    if flag_s==0:
        scenicSpot=[i for i in scenicSpot if df.loc[i,'type1']!='SC']
    elif flag_s==1:
        scenicSpot=[i for i in scenicSpot if df.loc[i,'type1']!='SN']
    elif flag_s==2:
        scenicSpot=[i for i in scenicSpot if df.loc[i,'type1']!='SP']
    scenicSpot=list(set(scenicSpot)-set(Tour))#去除已经走过的点
    
    #distance 越低越好
    next_scenicSpot = np.argsort(dis_matrix[Tour[-1], scenicSpot])#距离最短的序列
    scenicSpot=[scenicSpot[i] for i in next_scenicSpot]#按照距离最短排序
    
    temp=[]#根据分数排序，再根据分数分组
    #score
#     for idx,val in df.loc[scenicSpot,:].sort_values('score',ascending=False).groupby('score',sort=False):
#         next_point=np.argsort(dis_matrix[Tour[-1],val.index])#每个分数段内的景点根据距离排序
#         temp+=val.index[next_point].tolist()
#     scenicSpot=temp

#     stay_time 越高越好
#     for idx,val in df.loc[scenicSpot,:].sort_values('stay_time',ascending=False).groupby('stay_time',sort=False):
#         next_point=np.argsort(dis_matrix[Tour[-1],val.index])#每个分数段内的景点根据距离排序
#         temp+=val.index[next_point].tolist()
#     scenicSpot=temp

#     cost  越高越好
#     for idx,val in df.loc[scenicSpot,:].sort_values('cost',ascending=False).groupby('cost',sort=False):
#         next_point=np.argsort(dis_matrix[Tour[-1],val.index])#每个分数段内的景点根据距离排序
#         temp+=val.index[next_point].tolist()
#     scenicSpot=temp

    for i in scenicSpot:
        val=[get_minus(k) for k in df.loc[i,'open_time'][week%7].split('-')]
        arrive=time+time_matrix[Tour[-1],i]
        leave=arrive+df.loc[i,'stay_time']
        #早上出门+路程时间在第一个景点开放时间之内
        if  val[0]<=arrive and leave<=val[1]:
            time=leave
            Tour.append(i)
            flag_s=['SC','SN','SP'].index(df.loc[i,'type1'])

            return Tour,time,flag_s
    #循环中，没有可行解，解除类型约束，即令flag_s=-1
    flag_s=-1
    return Tour,time,flag_s

In [36]:
def dayTrip(Tour:list,name:str,week:int,time:int,flag_h,flag_r,flag_s):
#     flag_s=-1#循环中，没有可行解，解除类型约束，即令flag_s=-1
    h = Tour[-1]
    start = perf_counter()
    while True:
        if time>AM_End:#时间超出了早上时间11:30
            if  flag_h==1:
                time+=50#直接留一小时在酒店吃午饭
                flag_h=0
            else:
                Tour,time,flag_r=restaurantSelection(Tour,name,week,time,flag_r)
            while True:
                Tour,time,flag_s=scenicSpotSelection(Tour,name,week,time,flag_s)
                if perf_counter()-start>4:#运行时间超过4秒，就是陷入了死循环，跳出
                    return False,False,False,False
                if time>PM_End:
                    time+=time_matrix[Tour[-1], h]
                    Tour.append(h)
                    return Tour,flag_h,flag_r,flag_s
        Tour,time,flag_s=scenicSpotSelection(Tour,name,week,time,flag_s)
        if perf_counter()-start>4:#运行时间超过4秒，就是陷入了死循环，跳出
            return False,False,False,False

# 城际间

In [37]:
def InterCity(fromto,h1,h2,week,AM_Start):
    c_c=[]
    aa=df_polyline.loc[df_polyline.city==fromto,'outerCity'].values[0]
    for i in aa:
        if feasible([h1,i,h2],week%7,get_time(AM_Start)):
            c_c.append([h1,i,h2])
    if c_c==[]:
        return [h2]
    a=np.argsort([calcTime(i) for i in c_c])#按离开时间升序排列
    return c_c[a[-1]][1:] #时间最晚的,不包含第一个酒店

# TotalTour

In [38]:
def TotalTour(Tour,week,flag_h,flag_r,flag_s,config,time):
    # 70分钟从机场到酒店加上休整时间 10分钟从机场到酒店,60分钟酒店休整
    Tour,flag_h,flag_r,flag_s=dayTrip(Tour,config.loc[0,'city'],week,time,flag_h,flag_r,flag_s) 
    week+=1
    #第一个城市
    for i in range(config.loc[0,'day']-1):
        Tour,flag_h,flag_r,flag_s=dayTrip(Tour,config.loc[0,'city'],week,AM_Start,flag_h,flag_r,flag_s)
        week+=1
    for i in range(len(config)-1):
        # 城市间
        fromto=config.loc[i,'city']+'_'+config.loc[i+1,'city']
        Tour+=InterCity(fromto,config.loc[i,'hotel'],config.loc[i+1,'hotel'],week,AM_Start)
        #下个城市的半天
        arr=splitTour(Tour)
        if calcTime(arr[-1])>AM_End:#早上到下个城市酒店的到达时间+60分钟的酒店入住整理时间 大于 11:30 就在酒店酒店吃饭
            flag_h=1#确定中午在酒店吃
        Tour,flag_h,flag_r,flag_s=dayTrip(Tour,config.loc[i+1,'city'],week, calcTime(arr[-1]),flag_h,flag_r,flag_s)
        week+=1
        #下个城市直接开始
        for j in range(config.loc[i+1,'day']-1):
            Tour,flag_h,flag_r,flag_s=dayTrip(Tour,config.loc[i+1,'city'],week,AM_Start,flag_h,flag_r,flag_s)
            week+=1
    return Tour,week,flag_h,flag_r,flag_s

In [39]:

config=pd.DataFrame(columns=['city','day','hotel'])
for i in ['Christchurch','Geraldine','Oamaru','Dunedin']:
    temp=recommend.loc[recommend.city==i,['city','day','hotel']]
    config=pd.concat([config,temp])
config.reset_index(drop=True,inplace=True)

Tour=[config.loc[0,'hotel']]
week=0
flag_h=1 
flag_s=-1
flag_r=-1

print(config)

Tour,week,flag_h,flag_r,flag_s=TotalTour(Tour,week,flag_h,flag_r,flag_s,config,AM_Start)
print(Tour)

           city day hotel
0  Christchurch   5    57
1     Geraldine   1   130
2        Oamaru   2   275
3       Dunedin   4   103
[57, 746, 695, 738, 754, 735, 57, 745, 713, 697, 1710, 715, 759, 57, 765, 739, 1700, 712, 719, 57, 747, 727, 1706, 704, 742, 57, 725, 709, 1682, 702, 728, 750, 57, 740, 130, 891, 888, 130, 894, 275, 1158, 1171, 1154, 275, 1164, 1163, 1993, 1161, 1168, 1160, 275, 1151, 103, 865, 834, 838, 103, 847, 837, 856, 1762, 830, 864, 103, 836, 850, 828, 1761, 853, 849, 103, 869, 845, 843, 1766, 882, 827, 858, 103]


# 南北岛结合

## 入境

In [40]:
#HKG-AKL 21:00-11:30(14个半小时)

## 北岛->南岛

In [41]:
# 70分钟从机场到酒店加上休整时间
# 10分钟从机场到酒店,60分钟酒店休整,50分钟酒店吃饭
'Auckland->Tauranga->Rotorua->飞机->Christchurch->Auckland'

''' Auckland->Tauranga->Rotorua '''
print(f'Journey Start:Hongkong->Auckland 21:00-11:30')
# 'Auckland->HongKong 08:55-15:10'

config=pd.DataFrame(columns=['city','day','hotel'])
for i in ['Auckland','Tauranga','Rotorua']:
    temp=recommend.loc[recommend.city==i,['city','day','hotel']]
    config=pd.concat([config,temp])
config.reset_index(drop=True,inplace=True)
print(config)
Tour=[config.loc[0,'hotel']]
week=0
flag_h=1 #在酒店就餐
flag_s=-1
flag_r=-1
arrTime='11:30'
start_time=get_minus(arrTime)+70  #70分钟从机场到酒店加上休整时间
Tour,week,flag_h,flag_r,flag_s=TotalTour(Tour,week,flag_h,flag_r,flag_s,config,start_time)
print(Tour)
# Tour1=[11, 595, 564, 571, 11, 636, 380, 1327, 1350, 380, 1343, 1351, 2126, 1355, 1340, 1347, 380, 1349, 339, 1275, 1284, 1267, 1252, 339, 1285, 1278, 1270, 2083, 1274, 1260, 1257, 339, 1265, 1286, 1266, 2070, 1288, 1251, 1289, 339]
print(feasible(Tour, 0,'13:30'))   #get_time(get_minus(arrTime)+70+50)=='13:30'  50分钟酒店就餐
dailySchedule(Tour, 0,'13:30')

Journey Start:Hongkong->Auckland 21:00-11:30
       city day hotel
0  Auckland   1    11
1  Tauranga   2   380
2   Rotorua   3   339
[11, 595, 564, 571, 11, 636, 380, 1327, 1350, 380, 1343, 1351, 2126, 1355, 1340, 1347, 380, 1349, 339, 1275, 1284, 1267, 1252, 339, 1285, 1278, 1270, 2083, 1274, 1260, 1257, 339, 1265, 1286, 1266, 2070, 1288, 1251, 1289, 339]
True


,city,name,score,cost,type1,type2,open_time,drive_time,arrive_time,stay_time,leave_time
11,Auckland,Eden Park Bed And Breakfast Inn,8.5,67,0,0,00:00-24:00,0,00:00,60,13:30
595,Auckland,Mobile Art Gallery,5.0,117,SC,SC1,00:00-24:00,6,13:36,130,15:46
564,Auckland,awesomeNZ,5.0,55,SP,SP6,06:30-18:00,9,15:55,70,17:05
571,Auckland,House Of Whiskey Ltd,5.0,122,SC,SC3,00:00-24:00,6,17:11,88,18:39
11,Auckland,Eden Park Bed And Breakfast Inn,8.5,67,0,0,00:00-24:00,14,18:53,60,08:00
636,Auckland,Mt William Walkway,4.5,133,SN,SN4,00:00-24:00,56,08:56,91,10:27
380,Tauranga,850 Cameron Motel,9.4,56,0,0,00:00-24:00,137,12:44,60,14:34
1327,Tauranga,Waikareao Estuary,4.5,97,SN,SN2,00:00-24:00,7,14:41,119,16:40
1350,Tauranga,Brain Watkins House Museum (Tauranga Historica...,3.5,100,SC,SC1,00:00-24:00,2,16:42,150,19:12
380,Tauranga,850 Cameron Motel,9.4,56,0,0,00:00-24:00,8,19:20,60,08:00


In [42]:
''' Rotorua->Christchurch '''
print(f'Journey Start:Rotorua->Christchurch 11:50-13:45')
config=pd.DataFrame(columns=['city','day','hotel'])
for i in ['Christchurch']:
    temp=recommend.loc[recommend.city==i,['city','day','hotel']]
    config=pd.concat([config,temp])
config.reset_index(drop=True,inplace=True)
print(config)
Tour=[config.loc[0,'hotel']]
arrTime='13:45'
start_time=get_minus(arrTime)+70  #70分钟从机场到酒店加上休整时间
if get_minus(arrTime)>AM_End:
    flag_h=1
Tour,week,flag_h,flag_r,flag_s=TotalTour(Tour,week,flag_h,flag_r,flag_s,config,start_time)
print(Tour)
# Tour2=[57, 754, 735, 57, 695, 738, 1700, 697, 715, 745, 57, 746, 722, 1706, 747, 704, 57, 765, 739, 1690, 700, 712, 57, 719, 709, 1682, 702, 742, 732, 57]
print(feasible(Tour, 6,'14:55'))   #get_time(get_minus(arrTime)+70)=='14:55' 飞机餐
dailySchedule(Tour, 6,'14:55')


Journey Start:Rotorua->Christchurch 11:50-13:45
           city day hotel
0  Christchurch   5    57
[57, 754, 735, 57, 695, 738, 1700, 697, 715, 745, 57, 746, 722, 1706, 747, 704, 57, 765, 739, 1690, 700, 712, 57, 719, 709, 1682, 702, 742, 732, 57]
True


,city,name,score,cost,type1,type2,open_time,drive_time,arrive_time,stay_time,leave_time
57,Christchurch,Sudima Christchurch City,9.4,80,0,0,00:00-24:00,0,00:00,60,14:55
754,Christchurch,Millbrook Reserve,5.0,144,SN,SN1,00:00-24:00,4,14:59,84,16:23
735,Christchurch,Hagley Oval,4.5,133,SC,SC1,00:00-24:00,5,16:28,120,18:28
57,Christchurch,Sudima Christchurch City,9.4,80,0,0,00:00-24:00,5,18:33,60,08:00
695,Christchurch,Margaret Mahy Family Playground,5.0,118,SP,SP2,00:00-24:00,5,08:05,116,10:01
738,Christchurch,The Piano,4.5,146,SC,SC1,00:00-24:00,3,10:04,130,12:14
1700,Christchurch,The Craft Embassy,4.5,52,RN,新西兰,10:00-16:00,3,12:17,50,13:07
697,Christchurch,Riccarton House & Bush,4.5,82,SN,SN1,09:00-16:00,9,13:16,73,14:29
715,Christchurch,Christchurch Farmers' Market,4.5,100,SC,SC3,00:00-24:00,1,14:30,112,16:22
745,Christchurch,Addington Raceway,4.5,101,SP,SP6,00:00-24:00,6,16:28,119,18:27


## 南岛->北岛

In [43]:
'''Auckland->飞机->Queenstown->Rotorua->飞机->Auckland'''
print(f'Hongkong->Auckland 21:00-11:30    Auckland->Queenstown 12:00-13:55')

config=pd.DataFrame(columns=['city','day','hotel'])
for i in ['Queenstown','Dunedin']:
    temp=recommend.loc[recommend.city==i,['city','day','hotel']]
    config=pd.concat([config,temp])
config.reset_index(drop=True,inplace=True)
print(config)
Tour=[config.loc[0,'hotel']]
week=0
flag_h=1
flag_s=-1
flag_r=-1
arrTime='13:55'
start_time=get_minus(arrTime)+70  #70分钟从机场到酒店加上休整时间
Tour,week,flag_h,flag_r,flag_s=TotalTour(Tour,week,flag_h,flag_r,flag_s,config,start_time)
print(Tour)

Hongkong->Auckland 21:00-11:30    Auckland->Queenstown 12:00-13:55
         city day hotel
0  Queenstown   3   321
1     Dunedin   4   103
[321, 1202, 321, 1210, 1236, 2050, 1219, 1201, 1240, 321, 1217, 1228, 1242, 2056, 1208, 1212, 321, 1232, 103, 865, 839, 103, 837, 836, 850, 1759, 830, 864, 845, 103, 838, 842, 1756, 853, 831, 869, 103, 847, 843, 860, 1766, 882, 827, 858, 103]


In [44]:
# Tour3=[321, 1202, 321, 1210, 1236, 2050, 1219, 1201, 1240, 321, 1217, 1228, 1242, 2056, 1208, 1212, 321, 1232, 103, 865, 839, 103, 837, 836, 850, 1759, 830, 864, 845, 103, 838, 842, 1756, 853, 831, 869, 103, 847, 843, 860, 1766, 882, 827, 858, 103]
print(feasible(Tour, 0 ,get_time(get_minus(arrTime)+70)))
dailySchedule(Tour, 0,get_time(get_minus(arrTime)+70))

True


,city,name,score,cost,type1,type2,open_time,drive_time,arrive_time,stay_time,leave_time
321,Queenstown,Queenstown Park Boutique Hotel,9.4,140,0,0,00:00-24:00,0,00:00,60,15:05
1202,Queenstown,Queenstown Trail,5.0,84,SN,SN4,00:00-24:00,1,15:06,115,17:01
321,Queenstown,Queenstown Park Boutique Hotel,9.4,140,0,0,00:00-24:00,1,17:02,60,08:00
1210,Queenstown,Queenstown Cemetery,4.0,89,SC,SC1,00:00-24:00,3,08:03,170,10:53
1236,Queenstown,Queenstown Ferries,4.0,95,SP,SP2,00:00-24:00,5,10:58,101,12:39
2050,Queenstown,The Taj Indian Kitchen,4.5,84,RA,亚洲菜,12:00-15:00,0,12:39,50,13:29
1219,Queenstown,Moa Statue,4.0,136,SC,SC4,00:00-24:00,1,13:30,71,14:41
1201,Queenstown,Queenstown Hill,4.5,73,SN,SN4,00:00-24:00,6,14:47,86,16:13
1240,Queenstown,St. Joseph Catholic Church,4.0,87,SC,SC1,00:00-24:00,4,16:17,170,19:07
321,Queenstown,Queenstown Park Boutique Hotel,9.4,140,0,0,00:00-24:00,3,19:10,60,08:00


In [45]:
print(f'Dunedin->Wellington 10:50-12:05')
# 70分钟从机场到酒店加上休整时间    
# 30分钟从机场到酒店,40分钟酒店休整,50分钟吃饭

config=pd.DataFrame(columns=['city','day','hotel'])
config.loc[0]=['Wellington',4,450]
print(config)
if get_minus(arrTime)>AM_End:
    flag_h=1
Tour=[config.loc[0,'hotel']]
arrTime='12:05'
start_time=get_minus(arrTime)+70  #70分钟从机场到酒店加上休整时间
Tour,week,flag_h,flag_r,flag_s=TotalTour(Tour,week,flag_h,flag_r,flag_s,config,start_time)
print(Tour)

Dunedin->Wellington 10:50-12:05
         city  day  hotel
0  Wellington    4    450
[450, 1431, 1481, 450, 1500, 1468, 2207, 1435, 1467, 450, 1476, 1493, 2206, 1428, 1421, 1472, 450, 1454, 1414, 2203, 1502, 1439, 450]


In [46]:
# Tour4=[450, 1431, 1481, 450, 1500, 1468, 2207, 1435, 1467, 450, 1476, 1493, 2206, 1428, 1421, 1472, 450, 1454, 1414, 2203, 1502, 1439, 450]
print(feasible(Tour, 7 ,get_time(get_minus(arrTime)+120)))
dailySchedule(Tour, 7,get_time(get_minus(arrTime)+120))

True


,city,name,score,cost,type1,type2,open_time,drive_time,arrive_time,stay_time,leave_time
450,Wellington,Doubletree By Hilton Wellington,8.9,79,0,0,00:00-24:00,0,00:00,60,14:05
1431,Wellington,Wellington Cable Car,4.5,96,SP,SP2,07:30-20:00,4,14:09,101,15:50
1481,Wellington,Woman of Words,5.0,117,SC,SC1,00:00-24:00,3,15:53,150,18:23
450,Wellington,Doubletree By Hilton Wellington,8.9,79,0,0,00:00-24:00,2,18:25,60,08:00
1500,Wellington,The Phantom of the Opera,4.5,76,SP,SP6,00:00-24:00,6,08:06,98,09:44
1468,Wellington,Kura Art Gallery,4.5,119,SC,SC1,00:00-24:00,2,09:46,160,12:26
2207,Wellington,Rock Yard Restaurant,4.5,80,RA,亚洲菜,10:00-16:00,0,12:26,50,13:16
1435,Wellington,East By West Ferries,4.5,115,SP,SP2,09:00-19:00,4,13:20,93,14:53
1467,Wellington,Paddy the Wanderer Monument,4.5,112,SC,SC1,00:00-24:00,2,14:55,180,17:55
450,Wellington,Doubletree By Hilton Wellington,8.9,79,0,0,00:00-24:00,5,18:00,60,08:00
